In [1]:
import json

def load_batch(K=50, data_path='maps.json'):
    data = json.load(open(data_path))
    if K > len(data):
        raise IndexError("K should be no greater than size of dataset. size=" + str(len(data)))
    objects = list(map(lambda x: x[0], data[:K]))
    feats = list(map(lambda x: x[1], data[:K]))
    return objects, feats

In [2]:
raw_Y, raw_X = load_batch(3)

In [6]:
X = raw_X

music_len = max(len(X[i]) for i in range(len(X)))
X_len = 65
Y_len = 4
num_hidden = 200
batch_size = 1
no_of_batches = int(len(X) / batch_size)
epoch = 10

Y = [[[0, 0, 0, 0] for j in range(music_len)] for i in range(len(X))]
PX = [[0.5] * music_len for i in range(len(X))]
PY = [[0.5] * music_len for i in range(len(X))]
for i in range(len(X)):
    for j in range(music_len):
        if j < len(raw_Y[i]):
            Y[i][j][int(raw_Y[i][j][2])] = 1
            PX[i][j] = raw_Y[i][j][0]
            PY[i][j] = raw_Y[i][j][1]
        else:
            X[i].append([0]*X_len)
print len(PX), len(PX[0])

3 338


In [15]:
import tensorflow as tf
data = tf.placeholder(tf.float32, [None, music_len, X_len])
target = tf.placeholder(tf.float32, [None, music_len, Y_len])
target_PX = tf.placeholder(tf.float32, [None, music_len])
target_PY = tf.placeholder(tf.float32, [None, music_len])

dfm = {data:X, target:Y, target_PX:PX, target_PY:PY}

cell = tf.nn.rnn_cell.LSTMCell(num_hidden, state_is_tuple=True)
output, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)
# output: [batch, length, feature]

In [16]:
weight = tf.Variable(tf.truncated_normal([num_hidden, Y_len]))
bias = tf.Variable(tf.constant(0.0, shape=[Y_len]))

weight_PX = tf.Variable(tf.truncated_normal([num_hidden, 1]))
bias_PX = tf.Variable(tf.constant(0.5, shape=[1]))

weight_PY = tf.Variable(tf.truncated_normal([num_hidden, 1]))
bias_PY = tf.Variable(tf.constant(0.5, shape=[1]))

In [17]:
val = tf.reshape(output, [-1, num_hidden])
tar = tf.reshape(target, [-1, Y_len])
tar_PX = tf.reshape(target_PX, [-1])
tar_PY = tf.reshape(target_PY, [-1])

preds = tf.matmul(val, weight) + bias
preds_PX = tf.sigmoid(tf.reshape(tf.matmul(val, weight_PX) + bias_PX, [-1]))
preds_PY = tf.sigmoid(tf.reshape(tf.matmul(val, weight_PY) + bias_PY, [-1]))

In [18]:
cross_ens = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(preds, tar))
l2_PX = tf.reduce_mean(tf.nn.l2_loss(preds_PX - tar_PX))
l2_PY = tf.reduce_mean(tf.nn.l2_loss(preds_PY - tar_PY))

losses = 100 * cross_ens + l2_PX + l2_PY

In [19]:
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(losses)

In [20]:
mistakes = tf.not_equal(tf.argmax(tar, 1), tf.argmax(preds, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

In [35]:
# ps, ps_PX, ps_PY = sess.run((preds, preds_PX, preds_PY), {data:X[:1], target:Y[:1], target_PX: PX[:1], target_PY: PY[:1]})

In [21]:
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

In [30]:
for i in range(100):
    ptr = 0
    for j in range(no_of_batches):
        inp, out, out_PX, out_PY = X[ptr:ptr+batch_size], Y[ptr:ptr+batch_size], PX[ptr:ptr+batch_size], PY[ptr:ptr+batch_size]
        ptr += batch_size
        sess.run(minimize, {data: inp, target: out, target_PX: out_PX, target_PY: out_PY})
    if i % 10 == 0:
        incor = sess.run(error, dfm)
        loss = sess.run(losses, dfm)
        cross = sess.run(cross_ens, dfm)
        lx = sess.run(l2_PX, dfm)
        ly = sess.run(l2_PY, dfm)
        print('Epoch {:2d} loss {:3.6f}'.format(i + 1, loss))
        print('         error {:3.6f}% cross {:3.6f} lx {:3.6f} ly {:3.6f}'.format(incor*100, cross, lx, ly))

Epoch  1 loss 29.229641
         error 16.074951% cross 0.001096 lx 2.926918 ly 26.193123
Epoch 11 loss 28.292675
         error 16.074951% cross 0.001001 lx 2.453407 ly 25.739164
Epoch 21 loss 27.201542
         error 16.074951% cross 0.000922 lx 1.958313 ly 25.151060
Epoch 31 loss 28.499416
         error 16.074951% cross 0.000872 lx 2.208383 ly 26.203846
Epoch 41 loss 26.919376
         error 16.074951% cross 0.000809 lx 1.792536 ly 25.045916
Epoch 51 loss 26.160730
         error 16.074951% cross 0.000772 lx 1.720324 ly 24.363186
Epoch 61 loss 25.829704
         error 15.976331% cross 0.000709 lx 1.720148 ly 24.038685
Epoch 71 loss 25.418091
         error 16.074951% cross 0.000663 lx 1.522245 ly 23.829502
Epoch 81 loss 25.146862
         error 16.074951% cross 0.000624 lx 1.563083 ly 23.521416
Epoch 91 loss 24.961935
         error 16.074951% cross 0.000600 lx 1.471150 ly 23.430817


In [34]:
sess.close()